In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from config import gkey
from haversine import haversine
from pprint import pprint

# needed to clear output below
from IPython.display import clear_output

In [3]:
# load standard prepared data fail
fire_data = pd.read_csv("Fire.csv")
fire_data.head(1)

,index,ID,DATE,TIME,ZIP,BATTALION,SHIFT,COUNCIL DISTRICT,STRUCTURE TYPE,BUILDING STORIES,...,STRUCTURE LOSS,CONTENTS LOSS,TOTAL SAVED,PERCENTAGE LOSS,SPRINKLER SYSTEM,ALARM SYSTEM,ADDRESS,Lat,Lng,TOTAL LOSS
0,0,17961.0,10/29/2019,08:21 PM,75216.0,6,A,4.0,DWELLING,1.0,...,-5000.0,0.0,161370.0,3.01,NaN,NaN,1325 S MARSALIS AVE,32.732107,-96.814783,5000.0


In [4]:
dallas_coords_df = pd.read_csv("data/Dallas_coords_grid.csv")
dallas_coords_df.head(1)

,Lat,Lng,Dist to 1st Station (mi),Dist to 2nd Station (mi)
0,33.01025,-96.481611,2.21,2.76


In [4]:
dallas_coords_df.to_csv(r"data/Dallas_coords_grid.csv", index=False)

In [5]:
# Load fire stations lat/long

fire_stations_df = pd.read_csv('data/fire_stations.csv')
fire_stations_df.head(2)

,Address,Lat,Lng
0,1901 Irving Blvd.,32.794005,-96.827024
1,4211 Northhaven Road,32.902599,-96.837838


In [6]:
#Search for 2 nearest fire stations, find latitude and longitude
places_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"



dist0 = []
dist1 = []
length = len(fire_data)
status = ""

for index, row in fire_data.iterrows():
    
    # store current lat long
    lat = row['Lat']
    lng = row['Lng']
    latlng = str(lat) + ',' + str(lng)
    
    params = {
        "location": latlng,
        "key": gkey,
        "type": "fire_station",
        'rankby': 'distance'
    }
    
    try:
        geo_data = requests.get(places_url, params=params).json()
        
        # record lat & long for first two results
        lat0 = geo_data["results"][0]["geometry"]["location"]["lat"]
        lng0 = geo_data["results"][0]["geometry"]["location"]["lng"]
        lat1 = geo_data["results"][1]["geometry"]["location"]["lat"]
        lng1 = geo_data["results"][1]["geometry"]["location"]["lng"]
        if (geo_data['status'] == 'INVALID_REQUEST'):
            status = "Invalid Request."
        else:
            status = 'found.'
        
        dist0.append(haversine(lat,lng,lat0,lng0,'mi'))
        dist1.append(haversine(lat,lng,lat1,lng1,'mi'))
        
    except:
        lat0 = None
        lng0 = None
        lat1 = None
        lng1 = None
        status = 'error.'
        dist0.append(None)
        dist1.append(None)
    
    # Show status while running, but without the clutter.
    clear_output(wait=True)
    print(f"{index+1} of {length}: {status}  Dist0: {dist0[index]}, Dist1: {dist1[index]}")
    
    
    

621 of 621: found.  Dist0: 1.1872932191257024, Dist1: 1.7557976327439737


In [7]:
# load new data into dataframe
fire_data['Dist to 1st Station (mi)'] = dist0
fire_data['Dist to 2nd Station (mi)'] = dist1

# round data to 2 decimal points
fire_data['Dist to 1st Station (mi)'] = fire_data['Dist to 1st Station (mi)'].round(2)
fire_data['Dist to 2nd Station (mi)'] = fire_data['Dist to 2nd Station (mi)'].round(2)
fire_data.head(10)

,index,ID,DATE,TIME,ZIP,BATTALION,SHIFT,COUNCIL DISTRICT,STRUCTURE TYPE,BUILDING STORIES,...,TOTAL SAVED,PERCENTAGE LOSS,SPRINKLER SYSTEM,ALARM SYSTEM,ADDRESS,Lat,Lng,TOTAL LOSS,Dist to 1st Station (mi),Dist to 2nd Station (mi)
0,0,17961.0,10/29/2019,08:21 PM,75216.0,6,A,4.0,DWELLING,1.0,...,161370.0,3.01,NaN,NaN,1325 S MARSALIS AVE,32.732107,-96.814783,5000.0,1.01,1.23
1,1,17962.0,10/30/2019,03:42 AM,75229.0,7,A,13.0,DWELLING,1.0,...,525470.0,1.87,NaN,NaN,10728 LATHROP DR,32.894248,-96.857108,10000.0,0.97,1.26
2,2,17963.0,10/30/2019,10:47 AM,75241.0,5,B,3.0,DWELLING,1.0,...,286000.0,6.54,NaN,NaN,6205 S R L THORNTON SERV,32.694543,-96.824020,20000.0,1.88,2.06
3,3,17964.0,10/30/2019,05:32 PM,75243.0,2,B,10.0,DWELLING,1.0,...,114710.0,9.47,NaN,NaN,12044 WILLOWDELL DR,32.913584,-96.764435,12000.0,0.27,1.43
4,4,17965.0,10/30/2019,11:03 AM,75212.0,9,B,6.0,DWELLING,1.0,...,29000.0,65.48,NaN,NaN,2212 LAPSLEY ST,32.769986,-96.908816,55000.0,0.46,2.37
5,5,17966.0,10/31/2019,09:25 PM,75212.0,9,C,6.0,DWELLING,1.0,...,34880.0,5.42,NaN,NaN,4007 ARANSAS ST,32.790290,-96.875248,2000.0,1.21,1.73
6,6,17967.0,10/31/2019,11:49 AM,75216.0,5,C,4.0,APARTMENTS,2.0,...,2595000.0,1.14,NaN,NaN,3540 WILHURT AVE,32.713743,-96.763392,30000.0,0.80,2.23
7,7,17968.0,10/31/2019,11:07 AM,75227.0,8,C,7.0,APARTMENTS,2.0,...,248000.0,0.48,NaN,NaN,4243 OSBORN RD,32.787776,-96.694828,1200.0,0.97,1.36
8,8,17969.0,10/31/2019,08:13 AM,75232.0,5,C,4.0,MOTEL,1.0,...,817800.0,0.85,NaN,NaN,5715 S R L THORNTON FRWY,32.676848,-96.823544,7000.0,1.03,2.05
9,9,17263.0,11/01/2019,01:53 AM,75228.0,8,C,7.0,APARTMENTS,2.0,...,4689000.0,0.13,NaN,NaN,3550 ST FRANCIS AVE,32.798165,-96.691340,6000.0,0.66,1.71


In [10]:
# Check data for missing results
fire_data.loc[fire_data['Dist to 2nd Station (mi)'].isna()]

,index,ID,DATE,TIME,ZIP,BATTALION,SHIFT,COUNCIL DISTRICT,STRUCTURE TYPE,BUILDING STORIES,...,TOTAL SAVED,PERCENTAGE LOSS,SPRINKLER SYSTEM,ALARM SYSTEM,ADDRESS,Lat,Lng,TOTAL LOSS,Dist to 1st Station (mi),Dist to 2nd Station (mi)
29,29,17275.0,11/08/2019,06:37 PM,75240.0,2,B,11.0,APARTMENTS,2.0,...,1970000.0,1.50,NO,YES,13740 MONTFORT DR,32.935232,-96.810022,30000,NaN,NaN
32,32,17280.0,11/09/2019,07:50 AM,75229.0,7,C,13.0,DWELLING,1.0,...,193080.0,0.52,NO,YES,3056 WILDFLOWER DR,32.901950,-96.875667,1000,NaN,NaN
40,40,17290.0,11/11/2019,09:20 PM,75287.0,2,B,12.0,APARTMENTS,3.0,...,2980000.0,1.00,YES,YES,3840 FRANKFORD RD,32.995566,-96.848082,30000,NaN,NaN
44,44,17295.0,11/12/2019,11:52 PM,75252.0,2,C,12.0,DWELLING,2.0,...,667407.0,1.48,NO,YES,18206 BRIGHTON GREEN,32.998482,-96.786273,10000,NaN,NaN
45,45,17296.0,11/12/2019,09:58 PM,75216.0,6,C,4.0,DWELLING,1.0,...,100000.0,7.41,NaN,NaN,1427 STELLA AVE,32.727821,-96.803137,8000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588,588,18423.0,09/06/2020,11:00 PM,75240.0,2,B,11.0,APARTMENTS,2.0,...,15003700.0,0.01,NO,YES,5854 PRESTON VIEW BLVD,32.934075,-96.807532,800,NaN,NaN
595,595,18475.0,09/24/2020,06:51 PM,75224.0,6,B,4.0,DWELLING,1.0,...,174670.0,1.96,NO,NO,2441 S LLEWELLYN AVE,32.717737,-96.831143,3500,NaN,NaN
598,598,18432.0,09/10/2020,05:23 PM,75243.0,4,C,10.0,DWELLING,1.0,...,137600.0,0.04,NO,YES,8328 CAMPANELLA DR,32.918196,-96.758976,50,NaN,NaN
604,604,18440.0,09/12/2020,01:43 PM,75225.0,4,C,13.0,COMMERCIAL,2.0,...,726499700.0,0.00,YES,YES,8687 N CENTRAL SERV SB,32.868487,-96.773600,300,NaN,NaN


In [8]:
# save back into group's fire csv file
fire_data.to_csv(r'Fire.csv', index = False)

In [9]:
# configuration guide: https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html

gmaps.configure(api_key=gkey)
dallas_coords = (32.7767, -96.7970)
figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px',
    
}
fig = gmaps.figure(layout=figure_layout)

coordinates = fire_data[['Lat','Lng']]

station_dist = fire_data['Dist to 1st Station (mi)']
heat_layer = gmaps.heatmap_layer(coordinates, weights=station_dist, dissipating=True, max_intensity=5, point_radius=20)
fig.add_layer(heat_layer)

fig
# existing fires and distance to closest fire station

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [10]:
gmaps.configure(api_key=gkey)
dallas_coords = (32.7767, 96.7970)
figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px',
    
}
fig = gmaps.figure(layout=figure_layout)

coordinates = fire_data[['Lat','Lng']]

station_dist = fire_data['Dist to 2nd Station (mi)']
heat_layer = gmaps.heatmap_layer(coordinates, weights=station_dist, dissipating=True, max_intensity=5, point_radius=20)
fig.add_layer(heat_layer)

fig
# existing fires and distance to 2nd-closest fire station

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [4]:
# grab geographical info for Dallas
places_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"

address = "Dallas, TX"

params = {
    "input": address.replace(" ", "+"),
    "inputtype": "textquery",
    "key": gkey
}

dallas_place_data = requests.get(places_url, params=params).json()
dallas_place_data

{'candidates': [{'place_id': 'ChIJS5dFe_cZTIYRj2dH9qSb7Lk'}], 'status': 'OK'}

In [5]:
# Get Place ID
dallas_place_id = dallas_place_data['candidates'][0]['place_id']
places_url = "https://maps.googleapis.com/maps/api/place/details/json"
params = {
    "place_id": dallas_place_id,
    "key": gkey
}
dallas_geo_data = requests.get(places_url, params=params).json()
pprint(dallas_geo_data, indent=4)

{   'html_attributions': [],
    'result': {   'address_components': [   {   'long_name': 'Dallas',
                                                'short_name': 'Dallas',
                                                'types': [   'locality',
                                                             'political']},
                                            {   'long_name': 'Dallas County',
                                                'short_name': 'Dallas County',
                                                'types': [   'administrative_area_level_2',
                                                             'political']},
                                            {   'long_name': 'Texas',
                                                'short_name': 'TX',
                                                'types': [   'administrative_area_level_1',
                                                             'political']},
                                            {   

In [6]:
viewport = dallas_geo_data['result']['geometry']['viewport']
viewport

{'northeast': {'lat': 33.0237920649461, 'lng': -96.46375805411309},
 'southwest': {'lat': 32.6175369971454, 'lng': -96.9993469629029}}

In [7]:
grid_resolution = 30
# grab corners of Dallas
lat_min = viewport['northeast']['lat']
lng_min = viewport['northeast']['lng']
lat_max = viewport['southwest']['lat']
lng_max = viewport['southwest']['lng']

lat_step = (lat_max - lat_min) / grid_resolution
lng_step = (lng_max - lng_min) / grid_resolution

# generate list of lats & lngs
square_lats = []
square_lngs = []

lat = lat_min
for i in range(grid_resolution + 1):
    square_lats.append(lat)
    lat += lat_step
    
lng = lng_min
for i in range(grid_resolution + 1):
    square_lngs.append(lng)
    lng += lng_step

# insert lats & lngs into coordinates list
square_coords = []
for i in range(grid_resolution + 1):
    for j in range(grid_resolution + 1):
        square_coords.append([square_lats[i],square_lngs[j]])
square_coords

[[33.0237920649461, -96.46375805411309],
 [33.0237920649461, -96.48161101773941],
 [33.0237920649461, -96.49946398136574],
 [33.0237920649461, -96.51731694499206],
 [33.0237920649461, -96.53516990861839],
 [33.0237920649461, -96.55302287224471],
 [33.0237920649461, -96.57087583587104],
 [33.0237920649461, -96.58872879949736],
 [33.0237920649461, -96.60658176312369],
 [33.0237920649461, -96.62443472675001],
 [33.0237920649461, -96.64228769037634],
 [33.0237920649461, -96.66014065400266],
 [33.0237920649461, -96.67799361762899],
 [33.0237920649461, -96.69584658125531],
 [33.0237920649461, -96.71369954488164],
 [33.0237920649461, -96.73155250850796],
 [33.0237920649461, -96.74940547213428],
 [33.0237920649461, -96.76725843576061],
 [33.0237920649461, -96.78511139938693],
 [33.0237920649461, -96.80296436301326],
 [33.0237920649461, -96.82081732663958],
 [33.0237920649461, -96.83867029026591],
 [33.0237920649461, -96.85652325389223],
 [33.0237920649461, -96.87437621751856],
 [33.02379206494

In [84]:
# Test coordinate list by placing markers on gmap
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

markers = gmaps.marker_layer(square_coords)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

In [70]:
# Explore reverse geocode
geo_url = 'https://maps.googleapis.com/maps/api/geocode/json'
params = {
    'key':gkey,
    'latlng': '32.888373709012534,-96.99934696290292',
    'result_type': 'locality'
}
response_json = requests.get(geo_url, params=params).json()
name = response_json['results'][0]['address_components'][0]['long_name']
name

'Irving'

In [8]:
geo_url = 'https://maps.googleapis.com/maps/api/geocode/json'
dallas_points = []
counter = 0
length = len(square_coords)
# test all points to see if it's in Dallas.  If it is, add it to dallas_points
for latlng in square_coords:
    params = {
    'key':gkey,
    'latlng': f'{latlng[0]},{latlng[1]}',
    'result_type': 'locality'
    }
    response_json = requests.get(geo_url, params=params).json()
    try:
        name = response_json['results'][0]['address_components'][0]['long_name']
        clear_output(wait=True)
        print(f'{counter+1} of {length}: {name}')
        if name == 'Dallas':
            dallas_points.append([latlng[0],latlng[1]])
    except:
        clear_output(wait=True)
        print(f'{counter+1} of {length}: Error: city not found.')
        
    counter +=1
    
# place dallas_points as Marker on Map
# manually check if shape looks like that found here: 
# https://www.randymajors.org/city-limits-on-google-maps?fips=48113&labels=show
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

markers = gmaps.marker_layer(dallas_points)
# Add the layer to the map
fig.add_layer(markers)
fig

961 of 961: Grand Prairie


Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

In [9]:
# Create dataframe which lists evenly spaced locations within City of Dallas

lats = []
lngs = []

for i in range(len(dallas_points)):
    lats.append(dallas_points[i][0])
    lngs.append(dallas_points[i][1])
    
# dallas_coords_dict['Lat'] = lats
# dallas_coords_dict['Lng'] = lngs
dallas_coords_df = pd.DataFrame({
    'Lat': lats,
    'Lng': lngs
})
dallas_coords_df.head()

,Lat,Lng
0,33.01025,-96.481611
1,33.01025,-96.802964
2,33.01025,-96.820817
3,33.01025,-96.838670
4,33.01025,-96.856523


In [14]:
#Search for 2 nearest fire stations, find latitude and longitude
places_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"



dist0 = []
dist1 = []
length = len(dallas_coords_df)
status = ""

for index, row in dallas_coords_df.iterrows():
    
    # store current lat long
    lat = row['Lat']
    lng = row['Lng']
    latlng = str(lat) + ',' + str(lng)
    
    params = {
        "location": latlng,
        "key": gkey,
        "type": "fire_station",
        'rankby': 'distance'
    }
    
    try:
        geo_data = requests.get(places_url, params=params).json()
        
        # record lat & long for first two results
        lat0 = geo_data["results"][0]["geometry"]["location"]["lat"]
        lng0 = geo_data["results"][0]["geometry"]["location"]["lng"]
        lat1 = geo_data["results"][1]["geometry"]["location"]["lat"]
        lng1 = geo_data["results"][1]["geometry"]["location"]["lng"]
        if (geo_data['status'] == 'INVALID_REQUEST'):
            status = "Invalid Request."
        else:
            status = 'found.'
        
        dist0.append(haversine(lat,lng,lat0,lng0,'mi'))
        dist1.append(haversine(lat,lng,lat1,lng1,'mi'))
        
    except:
        lat0 = None
        lng0 = None
        lat1 = None
        lng1 = None
        status = 'error.'
        dist0.append(None)
        dist1.append(None)
    
    # Show status while running, but without the clutter.
    clear_output(wait=True)
    print(f"{index+1} of {length}: {status}  Dist0: {dist0[index]}, Dist1: {dist1[index]}")
    
    
    

397 of 397: found.  Dist0: 0.9948604196150078, Dist1: 1.0455529965113115


In [ ]:
# load new data into dataframe
dallas_coords_df['Dist to 1st Station (mi)'] = dist0
dallas_coords_df['Dist to 2nd Station (mi)'] = dist1

# round data to 2 decimal points
dallas_coords_df['Dist to 1st Station (mi)'] = dallas_coords_df['Dist to 1st Station (mi)'].round(2)
dallas_coords_df['Dist to 2nd Station (mi)'] = dallas_coords_df['Dist to 2nd Station (mi)'].round(2)
dallas_coords_df_dropna = dallas_coords_df.dropna()


In [19]:
dallas_coords_df_dropna

,Lat,Lng,Dist to 1st Station (mi),Dist to 2nd Station (mi)
0,33.010250,-96.481611,2.21,2.76
1,33.010250,-96.802964,1.23,1.41
8,32.996708,-96.802964,1.09,1.44
11,32.996708,-96.856523,1.28,1.42
13,32.983167,-96.499464,2.56,2.66
...,...,...,...,...
392,32.631079,-96.731553,2.29,2.32
393,32.631079,-96.749405,1.67,1.70
394,32.631079,-96.767258,1.57,1.61
395,32.631079,-96.945788,0.89,0.89


In [11]:
# configuration guide: https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html

### heatmap
gmaps.configure(api_key=gkey)
dallas_coords = (32.7767, 96.7970)
figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px',
    
}
fig = gmaps.figure(layout=figure_layout)
dallas_coords_df_dropna = dallas_coords_df.dropna()
coordinates = dallas_coords_df_dropna[['Lat','Lng']]

station_dist = dallas_coords_df_dropna['Dist to 1st Station (mi)']
heat_layer = gmaps.heatmap_layer(coordinates, weights=station_dist, dissipating=True, max_intensity=4, point_radius=40)
fig.add_layer(heat_layer)

### markers
station_coords = []

for index, row in fire_stations_df.iterrows():
    station_coords.append([row['Lat'],row['Lng']])

markers = gmaps.marker_layer(station_coords)
fig.add_layer(markers)


fig
# City of Dallas borders heatmap, evenly spread.  Red indicates ~4 mi to 1st fire station.
# Ignore lake area.

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [12]:
# configuration guide: https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html

gmaps.configure(api_key=gkey)
dallas_coords = (32.7767, 96.7970)
figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px',
    
}
fig = gmaps.figure(layout=figure_layout)

coordinates = dallas_coords_df_dropna[['Lat','Lng']]

station_dist = dallas_coords_df_dropna['Dist to 2nd Station (mi)']
heat_layer = gmaps.heatmap_layer(coordinates, weights=station_dist, dissipating=True, max_intensity=5, point_radius=40)
fig.add_layer(heat_layer)

markers = gmaps.marker_layer(station_coords)
fig.add_layer(markers)

fig
# City of Dallas borders heatmap, evenly spread.  Red indicates ~5 mi to 2nd fire station.
# Ignore lake area.

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [14]:
# Load census data for even city points
dallas_census_df = pd.read_csv("data/Dallas_coords_census_data.csv")

# focus on percentage of households within income brackets
census_percentages = dallas_census_df.filter(like='Percentage')

census_percentages['Lat'] = dallas_census_df['Lat']
census_percentages['Lng'] = dallas_census_df['Lng']
census_percentages.head()

C:\Users\Rob\AppData\Local\Temp/ipykernel_27196/2069207386.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  census_percentages['Lat'] = dallas_census_df['Lat']
C:\Users\Rob\AppData\Local\Temp/ipykernel_27196/2069207386.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  census_percentages['Lng'] = dallas_census_df['Lng']


,"ACS Economics/Household income/Less than $10,000/Percentage","ACS Economics/Household income/$10,000 to $14,999/Percentage","ACS Economics/Household income/$15,000 to $19,999/Percentage","ACS Economics/Household income/$20,000 to $24,999/Percentage","ACS Economics/Household income/$25,000 to $29,999/Percentage","ACS Economics/Household income/$30,000 to $34,999/Percentage","ACS Economics/Household income/$35,000 to $39,999/Percentage","ACS Economics/Household income/$40,000 to $44,999/Percentage","ACS Economics/Household income/$45,000 to $49,999/Percentage","ACS Economics/Household income/$50,000 to $59,999/Percentage",...,"ACS Housing/Value of owner-occupied housing units/$250,000 to $299,999/Percentage","ACS Housing/Value of owner-occupied housing units/$300,000 to $399,999/Percentage","ACS Housing/Value of owner-occupied housing units/$400,000 to $499,999/Percentage","ACS Housing/Value of owner-occupied housing units/$500,000 to $749,999/Percentage","ACS Housing/Value of owner-occupied housing units/$750,000 to $999,999/Percentage","ACS Housing/Value of owner-occupied housing units/$1,000,000 to $1,499,999/Percentage","ACS Housing/Value of owner-occupied housing units/$1,500,000 to $1,999,999/Percentage","ACS Housing/Value of owner-occupied housing units/$2,000,000 or more/Percentage",Lat,Lng
0,0.000,0.007,0.038,0.017,0.024,0.000,0.000,0.011,0.05,0.071,...,0.275,0.176,0.062,0.000,0.000,0.000,0.0,0.0,33.01025,-96.481611
1,0.039,0.019,0.000,0.000,0.000,0.000,0.060,0.000,0.00,0.019,...,0.021,0.326,0.332,0.213,0.017,0.019,0.0,0.0,33.01025,-96.802964
2,0.097,0.000,0.000,0.062,0.025,0.083,0.088,0.041,0.07,0.107,...,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.0,33.01025,-96.820817
3,0.053,0.000,0.024,0.012,0.012,0.000,0.022,0.134,0.00,0.089,...,0.231,0.324,0.000,0.000,0.000,0.000,0.0,0.0,33.01025,-96.838670
4,0.059,0.023,0.045,0.047,0.059,0.178,0.058,0.099,0.12,0.036,...,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.0,33.01025,-96.856523


In [46]:
print(census_percentages.columns)

Index(['ACS Economics/Household income/Less than $10,000/Percentage',
       'ACS Economics/Household income/$10,000 to $14,999/Percentage',
       'ACS Economics/Household income/$15,000 to $19,999/Percentage',
       'ACS Economics/Household income/$20,000 to $24,999/Percentage',
       'ACS Economics/Household income/$25,000 to $29,999/Percentage',
       'ACS Economics/Household income/$30,000 to $34,999/Percentage',
       'ACS Economics/Household income/$35,000 to $39,999/Percentage',
       'ACS Economics/Household income/$40,000 to $44,999/Percentage',
       'ACS Economics/Household income/$45,000 to $49,999/Percentage',
       'ACS Economics/Household income/$50,000 to $59,999/Percentage',
       'ACS Economics/Household income/$60,000 to $74,999/Percentage',
       'ACS Economics/Household income/$75,000 to $99,999/Percentage',
       'ACS Economics/Household income/$100,000 to $124,999/Percentage',
       'ACS Economics/Household income/$125,000 to $149,999/Percentage',
   

C:\Users\Rob\AppData\Local\Temp/ipykernel_22248/2372235590.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  census_percentages['High Income'] = census_percentages['ACS Economics/Household income/$150,000 to $199,999/Percentage'] + \


In [15]:
census_percentages['High Income'] = census_percentages['ACS Economics/Household income/$150,000 to $199,999/Percentage'] + \
census_percentages['ACS Economics/Household income/$200,000 or more/Percentage'] + \
census_percentages['ACS Economics/Household income/$125,000 to $149,999/Percentage'] + \
census_percentages['ACS Economics/Household income/$100,000 to $124,999/Percentage']

census_percentages['Low Income'] = census_percentages['ACS Housing/Value of owner-occupied housing units/Less than $10,000/Percentage'] + \
census_percentages['ACS Housing/Value of owner-occupied housing units/$10,000 to $14,999/Percentage'] + \
census_percentages['ACS Housing/Value of owner-occupied housing units/$15,000 to $19,999/Percentage']

low_income_df = census_percentages[['Lat', 'Lng', 'Low Income']]

high_income_df = census_percentages[['Lat', 'Lng', 'High Income']]
high_income_df

C:\Users\Rob\AppData\Local\Temp/ipykernel_27196/497579538.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  census_percentages['High Income'] = census_percentages['ACS Economics/Household income/$150,000 to $199,999/Percentage'] + \
C:\Users\Rob\AppData\Local\Temp/ipykernel_27196/497579538.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  census_percentages['Low Income'] = census_percentages['ACS Housing/Value of owner-occupied housing units/Less than $10,000/Percentage'] + \


,Lat,Lng,High Income
0,33.010250,-96.481611,0.451
1,33.010250,-96.802964,0.723
2,33.010250,-96.820817,0.090
3,33.010250,-96.838670,0.485
4,33.010250,-96.856523,0.078
...,...,...,...
392,32.631079,-96.731553,0.000
393,32.631079,-96.749405,0.000
394,32.631079,-96.767258,0.000
395,32.631079,-96.945788,0.221


In [17]:
# configuration guide: https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html

gmaps.configure(api_key=gkey)
dallas_coords = (32.7767, -96.7970)
figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px',
    
}
fig = gmaps.figure(layout=figure_layout, zoom_level=11, center = dallas_coords)

coordinates1 = dallas_coords_df_dropna[['Lat','Lng']]

station_dist = dallas_coords_df_dropna['Dist to 1st Station (mi)']
colors1 = [(0,255,255,0.0),(255,255,0,0.5), (255,0,0,1)]
heat_layer1 = gmaps.heatmap_layer(coordinates1, weights=station_dist, dissipating=False, max_intensity=4, point_radius=.02, gradient=colors1)
fig.add_layer(heat_layer1)

coordinates2 = high_income_df[['Lat','Lng']]
colors2 = [(0,255,255,0.1),(0,190,128,0.3), (0,190,255,1)]
heat_layer2 = gmaps.heatmap_layer(coordinates2, weights=high_income_df['High Income'], dissipating=False, max_intensity=0.5,point_radius=.02, gradient=colors2)
fig.add_layer(heat_layer2)


fig
# City of Dallas borders heatmap, evenly spread.  Red indicates larger distance to 1st fire station.
# Blue indicates high income areas
# Ignore lake area.

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [18]:
# configuration guide: https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html

gmaps.configure(api_key=gkey)
dallas_coords = (32.7767, -96.7970)
figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px',
    
}
fig = gmaps.figure(layout=figure_layout, zoom_level=11, center = dallas_coords)

coordinates1 = dallas_coords_df_dropna[['Lat','Lng']]

station_dist = dallas_coords_df_dropna['Dist to 1st Station (mi)']
colors1 = [(0,255,255,0.0),(255,255,0,0.5), (255,0,0,1)]
heat_layer1 = gmaps.heatmap_layer(coordinates1, weights=station_dist, dissipating=False, max_intensity=4, point_radius=.02, gradient=colors1)
fig.add_layer(heat_layer1)

coordinates2 = low_income_df[['Lat','Lng']]
colors2 = [(0,255,255,0.1),(0,190,128,0.3), (0,190,255,1)]
heat_layer2 = gmaps.heatmap_layer(coordinates2, weights=low_income_df['Low Income'], dissipating=False, max_intensity=2.,point_radius=.02, gradient=colors2)
fig.add_layer(heat_layer2)


fig
# City of Dallas borders heatmap, evenly spread.  Red indicates larger distance to 1st fire station.
# Blue indicates low income areas
# Ignore lake area.

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))